In [56]:
import configparser
import os
import sys
from time import sleep
from uuid import uuid4

import numpy as np

import GCode
import GRBL

#

# Code:

In [73]:
# Poplar 1x4". Cut
BlockHeight = 68.0  # [mm]
BlockLength = 143  # [mm]
BlockThickness = 19.0  # [mm]
print("{} x {} x {}".format(BlockHeight, BlockLength, BlockThickness))


68.0 x 143 x 19.0


In [58]:
config = configparser.ConfigParser()
config["GRBL"] = dict()
config["GRBL"]["port"] = "/dev/cnc_3018"
cnc = GRBL.GRBL(**config["GRBL"])

cnc.reset()

(0, ['ok', '', "Grbl 1.1f ['$' for help]", 'ok'])

In [59]:
if "Alarm" in cnc.status:
    cnc.home()

In [60]:
cnc.init()

True

In [63]:
cnc.status

'<Idle|MPos:-255.900,-136.000,-1.000|Bf:15,127|FS:0,0>'

# Generate Program


In [64]:
def dot(X=10, Y=10, Z=0, power=1, dwell=1, **kwargs):
    program = GCode.GCode(**kwargs)
    
    program.G0(X=X, Y=Y, Z=Z)
    program.M3(S=power)
    program.G1(Z=Z, Y=Y, X=X)
    program.G4(P=dwell)
    program.M5()  # Laser. Off.

    return program

def dot_grid(X=(0, 10), Y=(0, 10), Z=0, n=3, power=1, dwell=1, **kwargs):
    XX = np.linspace(X[0], X[1], n, endpoint=True)
    YY = np.linspace(Y[0], Y[1], n, endpoint=True)
    XXX, YYY = np.meshgrid(XX, YY)
    program = GCode.GCode(**kwargs)
    
    for XXX_, YYY_ in zip(XXX, YYY):
        for X, Y in zip(XXX_, YYY_):
            program+=dot(X=X, Y=Y, Z=Z, power=power, dwell=dwell)
    return program

In [ ]:
dots

In [81]:

for idx, n in enumerate(range(3, 17, 1)):
    print("{} {} {}".format(idx*10, n, n))

0 3 3
10 4 4
20 5 5
30 6 6
40 7 7
50 8 8
60 9 9
70 10 10
80 11 11
90 12 12
100 13 13
110 14 14
120 15 15
130 16 16


In [82]:
dwells = np.linspace(0.5, 5, 5)

In [85]:
programs=list()
for idx3, dwell in enumerate([1]):
    for idx4, n in enumerate(range(3, 17, 1)):
        X=(idx4*10, (idx4+1)*10)
        Y=(10, 20)
        Z=-14
        program = dot_grid(X=X, Y=Y, Z=Z,n=n, power=power, dwell=dwell)
        programs.append(program)

In [86]:
programs

[<GCode>[cmds=45],
 <GCode>[cmds=80],
 <GCode>[cmds=125],
 <GCode>[cmds=180],
 <GCode>[cmds=245],
 <GCode>[cmds=320],
 <GCode>[cmds=405],
 <GCode>[cmds=500],
 <GCode>[cmds=605],
 <GCode>[cmds=720],
 <GCode>[cmds=845],
 <GCode>[cmds=980],
 <GCode>[cmds=1125],
 <GCode>[cmds=1280]]

In [ ]:
programs[0]

In [88]:
cnc.status

'<Idle|MPos:-255.900,-136.000,-1.000|Bf:15,127|FS:0,0>'

In [89]:
cnc.init()

True

In [ ]:
for idx, program in enumerate(programs):
    print("{}: ".format(idx), end="...")
    print(cnc.run(program))

0: ...29.153420209884644
1: ...49.34726667404175
2: ...76.23311734199524
3: ...108.95877170562744
4: ...124.29014897346497
5: ...158.26511192321777
6: ...198.8418197631836
7: ...244.13330698013306
8: ...293.9530816078186
9: ...348.5237305164337
10: ...407.628680229187
11: ...471.439325094223
12: ...539.8347897529602
13: ...